In [57]:
import bokeh.io
import pandas as pd
import numpy as np
import networkx as nx

from bokeh.io import show
from bokeh.plotting import Figure
from bokeh.models import Plot, Range1d, MultiLine, Circle, HoverTool, BoxZoomTool, ResetTool, PanTool, WheelZoomTool
from bokeh.models.graphs import from_networkx
from bokeh.palettes import Spectral4

bokeh.io.output_notebook()

Loading BokehJS ...

In [98]:
os.getcwd()

'/home/me/.netwulf'

In [96]:
df = pd.read_csv("data/fullevents.csv")

players = np.unique(df["DestinationPlayerID"][~pd.isnull(df.DestinationPlayerID)])

FileNotFoundError: [Errno 2] File data/fullevents.csv does not exist: 'data/fullevents.csv'

In [3]:
np.setxor1d(np.unique(df["OriginPlayerID"]), np.unique(df["DestinationPlayerID"][~pd.isnull(df["DestinationPlayerID"])])) #wtf

array(['Opponent5_D9'], dtype=object)

In [9]:
g = nx.Graph()
#g.add_nodes_from(players)

for i in players:
    if 'Husk' in i:
        c = Spectral4[1]
    else: c = Spectral4[0]
    
    g.add_node(i, color=c)

In [14]:
for index, row in df.iterrows():
    if not pd.isnull(row.DestinationPlayerID):
        g.add_edge(row.OriginPlayerID, row.DestinationPlayerID)

In [16]:
plot = Plot(plot_width=800, plot_height=600,
            x_range=Range1d(-1.1, 1.1), y_range=Range1d(-1.1, 1.1))
#plot.title.text = "Graph Interaction Demonstration"

node_hover_tool = HoverTool(tooltips=[("index", "@index")])
plot.add_tools(node_hover_tool, PanTool(), WheelZoomTool(), ResetTool())

graph_renderer = from_networkx(g, nx.spring_layout, scale=1, center=(0, 0))

graph_renderer.node_renderer.glyph = Circle(size=15, fill_color="color")
graph_renderer.edge_renderer.glyph = MultiLine(line_alpha=0.8, line_width=1)
plot.renderers.append(graph_renderer)

show(plot)


In [48]:
netwulf.visualize(g)

OSError: [Errno 98] Address already in use

In [54]:
df.EventOrigin_x, df.EventOrigin_y

(0          0.0
 1         34.0
 2         66.0
 3         22.0
 4         34.0
 5         59.0
 6         41.0
 7         36.0
 8         55.0
 9         43.0
 10        53.0
 11        69.0
 12        30.0
 13        65.0
 14        19.0
 15         5.0
 16        67.0
 17        33.0
 18        42.0
 19        36.0
 20        64.0
 21        30.0
 22        28.0
 23        72.0
 24        39.0
 25        73.0
 26        30.0
 27        31.0
 28        27.0
 29        36.0
 30        65.0
 31        34.0
 32        34.0
 33        52.0
 34        53.0
 35        48.0
 36        46.0
 37        56.0
 38        14.0
 39        11.0
 40         5.0
 41        53.0
 42         9.0
 43        38.0
 44        62.0
 45        41.0
 46        35.0
 47        65.0
 48        28.0
 49        68.0
 50        32.0
 51        25.0
 52        39.0
 53        61.0
 54        44.0
 55        56.0
 56        44.0
 57        59.0
 58         0.0
 59        89.0
 60        11.0
 61        74.0
 62     

In [86]:
f2 = Figure(title="Destination Events", plot_width=800, plot_height=600)
f2.grid.visible = False

idx = ~pd.isnull(df.EventDestination_x)

#heatmap, xedges, yedges = np.histogram2d(df.EventOrigin_x[idx], df.EventOrigin_y[idx], bins=50)
#extent = [xedges[0], xedges[-1], yedges[0], yedges[-1]]

r, bins = f2.hexbin(df.EventDestination_x[idx], df.EventDestination_y[idx], size=2)

f2.add_tools(HoverTool(
    tooltips=[("count", "@c"), ("(q,r)", "(@q, @r)")],
    mode="mouse", point_policy="follow_mouse", renderers=[r]
))

show(f2)

In [106]:
df[(df.OriginPlayerID == 'Opponent13_D1') & (df.DestinationPlayerID == 'Opponent16_D5')]

,MatchID,TeamID,OriginPlayerID,DestinationPlayerID,MatchPeriod,EventTime,EventType,EventSubType,EventOrigin_x,EventOrigin_y,EventDestination_x,EventDestination_y
51284,33,Opponent16,Opponent13_D1,Opponent16_D5,2H,1699.870116,Pass,Head pass,47.0,49.0,69.0,93.0
